In [1]:
#!!! Fix data structure for features_lst -> interpreted as string 
# should be list of str elems

In [1]:
# for colab compatibility
# !pip install shap
# !pip install ppscore
# !pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
%run data_load_wrapper.ipynb

In [3]:
# rip - bug1: feat_ind refers to local instead of global index ordering in function model_fit
# rip - bug2: stats dataframe is off by one for mse_test and train - now we have a shifted column
# 0th iteration replaced with full_set
#
# refactor1: get_metric
# refactor2: into class MetaReducer(dataset, imp_types, models) 
# refactor3: create a "smart" calculation execution for feature reduction
# we can pool the "votes" for feature reduction and only retrain the model once per vote class
# (feature to reduce), might save decent amount of time for large datasets 
# (cases where model training takes significantly longer than importance calculation)
# current: retrain the model every time for each {imp_type} - not as important right now, since
# model training in the worst case takes 3s, feature importance calculation is the bottleneck
""" 
dataset - sklearn_dataset object or tuple/list of tuples of (x, y)
imp_types - list of importance types
supported types: ['gini', 'pimp', 'shap', 'pps', 'pearson', 'spearman', 'kendall']
models - list of machine learning models from sklearn lib
[which ones? tbd]
debug = {False, True} display debug_info into the dataframe
return -> dataframe of reduction stats
""";

In [4]:
from tqdm.notebook import trange
import sklearn.datasets as data
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn import *
from sklearn.feature_selection import *
from sklearn.datasets import make_regression
from sklearn.utils import Bunch
import matplotlib as plt
import seaborn as sns
import shap
import numpy as np
import ppscore as pps
from datetime import datetime
import xgboost

In [5]:
def sklearn_to_df(dataset):
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    y = dataset.target
    return df, y

def get_feat_imp(imp_type, m, x_train, y_train, random_state):
    imp_type_dict = {
        'mutual_info' : 'mutual_info_regression(x_train, y_train)',
        'f_test' : 'f_regression(x_train, y_train)[0]',       
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state={random_state}).importances_mean""",
        'shap' : "abs(shap.TreeExplainer(m).shap_values(x_train).mean(axis=0))",
        
        'pps' : """(pps.predictors(x_train.join(y_train), 
                                'target', sorted=False, output='df').model_score
                                )""",
        
        'pearson' : "abs(x_train.join(y_train).corr(method='pearson').target[:-1])",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target[:-1])",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target[:-1])"
    }
#     print(eval(imp_type_dict.get(imp_type)))
    return eval(imp_type_dict.get(imp_type))

def reduce_dataset(x_train, x_test, drop_index):
    x_train = x_train.drop(x_train.columns[drop_index], axis=1)
    x_test = x_test.drop(x_test.columns[drop_index], axis=1)
    return x_train, x_test

def eval_reduced(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    reduce_stats = []
    columns_init = np.array(dataset.feature_names)
#     dataset_name = dataset.filename.split('/')[-1]
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit(x_train, y_train, 
                                                     x_test, y_test, m, imp_type, random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
    return reduce_stats

def model_fit(x_train, y_train, x_test, y_test, m, imp_type, random_state):
#     print(x_train.shape, y_train.shape)
#     refactor1 into function get_metric(type={train, test})
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.mean_squared_error(y_test, res_test)
    mse_train = metrics.mean_squared_error(y_train, res_train)
    # get the index for feature with lowest global feature importance
    imp = get_feat_imp(imp_type, m, x_train, y_train, random_state)
    argmin_fi_index = imp.argmin()
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

def eval_reduced_one_shot(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    
    reduce_stats = []
    columns_init = np.array(x_train.columns)
    m.set_params(random_state = random_state)
    m.fit(x_train, y_train)
    fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#     print(f'imp: {imp_type} \n model: {m} \n x_shape: {x_train.shape} \n y_shape: {y_train.shape}')
#     print(f'arr_scores: {fi_scores} \n shape: {fi_scores.shape}')
    
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit_one_shot(x_train, y_train, 
                                                     x_test, y_test, m, fi_scores, random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
#         print(global_index)
#         print(fi_scores)
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
#         print(f'iter {i}')
#         print(f'local_index: {drop_index}')
#         print(f'global_index: {global_index}, \n fi before: \n {fi_scores}')
#         print(fi_scores.shape)
        fi_scores = np.array(fi_scores)
        fi_scores = np.delete(fi_scores, drop_index)
#         print(f'global_index: {global_index}, \n fi after: \n {fi_scores}')              
    return reduce_stats

def model_fit_one_shot(x_train, y_train, x_test, y_test, m, fi_scores, 
                       random_state):
#     print(x_train.shape, y_train.shape)
#     refactor1 into function get_metric(type={train, test})
    m.set_params(random_state=random_state)
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.mean_squared_error(y_test, res_test)
    mse_train = metrics.mean_squared_error(y_train, res_train)
    # get the index for feature with lowest global feature importance
    imp = fi_scores
    argmin_fi_index = imp.argmin()
    
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

In [6]:
def get_stats(imp_types, models, datasets, debug=False, random_states=[0], dataset_states=[0]):
    results = []
    datasets_str = list(datasets.keys())
    datasets_data = datasets.items()
    random_state = 0
    for ds in trange(len(dataset_states), desc='Data state progress'):
        dataset_state = dataset_states[ds]
        for r in trange(len(random_states), desc='Model random state progress'):
            random_state = random_states[r]

            for j in trange(len(datasets_str), desc=f'Dataset progress'):
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df, y = sklearn_to_df(dataset)
                x_train, x_test = model_selection.train_test_split(df, test_size=0.3, 
                                                                   random_state=dataset_state)
                columns_init = x_train.columns
                y_train = pd.DataFrame(y[x_train.index], index=x_train.index, columns=['target'])
                y_test = pd.DataFrame(y[x_test.index], index=x_test.index, columns=['target'])

                for k in trange(len(models), desc=f'Model progress for dataset {j}'):
                    m = models[k]
                    m.set_params(random_state=random_state)
                    m.fit(x_train, y_train.target)
                    res_test = m.predict(x_test)
                    res_train = m.predict(x_train)
                    mse_test = metrics.mean_squared_error(y_test, res_test)
                    mse_train = metrics.mean_squared_error(y_train, res_train)

                    for i in trange(len(imp_types), desc=f'Importance type progress for model {k}'):
                        if imp_types[i] in ['shap', 'pimp', 'pearson', 'spearman', 'kendall',
                                            'pps', 'mutual_info', 'f_test']:
                            result = eval_reduced_one_shot(x_train, y_train, x_test, y_test, 
                                                           m, imp_types[i], dataset, dataset_str, 
                                                           random_state, dataset_state)
                        else:
                            result = eval_reduced(x_train, y_train.target, x_test, y_test, 
                                                  m, imp_types[i], dataset, dataset_str, 
                                                  random_state, dataset_state)
                        results.extend(result)
                df_res = pd.DataFrame(results, columns=['iteration', 'feat_ind', 'feat_name',
                                                    'mse_train', 'mse_test','model', 'imp_type', 
                                                    'dataset', 'features_lst', 
                                                    'random_state', 'data_split'])
        # shift features by 1 row, replace 0th iteration with 'full_set'
        df_res['dropped_feature'] = df_res['feat_name'].shift(1)
        df_res['dropped_feature'] = df_res['dropped_feature'].where(df_res['iteration'] != 0, 'full_set')
        if debug==True:
            repr_lst = ['iteration', 'dropped_feature', 'feat_ind', 'feat_name',
                        'mse_train', 'mse_test','model', 'imp_type', 
                        'dataset', 'features_lst','random_state', 'data_split']
        else:
            repr_lst = ['iteration','dropped_feature','mse_train',
         'mse_test','model','imp_type', 'dataset', 'random_state', 'data_split']
        df_res = df_res[repr_lst]
    return df_res

In [7]:
def coerce_synthethic_datasets(samples, features, percent_informative, random_state):
    data, target = make_regression(n_samples=samples, n_features=features, 
                                       n_informative=int(percent_informative*features),
                                       shuffle=False,
                                       random_state=random_state
                                      )
    feature_names = [f'x_{i}' for i in range(features)]
    # data, target, feature_names
    data_bunch = Bunch(
        data=data,
        target=target,
        feature_names=feature_names,
        informative_features=feature_names[:int(percent_informative*features)]
                        )
    return data_bunch

In [8]:
datasets = {}
imp_types = [
             'gini',
             'mutual_info',
             'f_test',
             'pimp', 
#              'shap', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]
n_jobs = -1
models = [
          tree.DecisionTreeRegressor(random_state=0), 
          ensemble.RandomForestRegressor(random_state=0, n_jobs=n_jobs, verbose=0), 
          xgboost.XGBRegressor(random_state=0, n_jobs=n_jobs, verbosity=0)
         ]

random_states = [i for i in range(1)]
dataset_states = [i for i in range(1)]

features = [10, 100]
samples = [10, 100, 1000, 10000]
# features = [100]
# samples = [10000]
synthethic_states = [i for i in range(1)]
percent_informative = 0.2

tuples = []
for i in features:
    for j in samples:
        if i != j:
            for k in synthethic_states:            
                tuples.append((i,j,k))
datasets = {}

In [9]:
for feat, samples, random_seed in tuples:
    datasets[f'regr_features_{feat}_samples_{samples}_seed_{random_seed}'] = coerce_synthethic_datasets(samples, feat, 0.2, random_seed)

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
random_state = 0
imp_types = [
             'gini',
             'mutual_info',
             'f_test',
             'pimp', 
#              'shap', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]

imp_type_dict = {
        'mutual_info' : 'mutual_info_regression(x_train, y_train)',
        'f_test' : 'f_regression(x_train, y_train)[0]',       
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state={random_state}).importances_mean""",
#         'shap' : "abs(shap.TreeExplainer(m).shap_values(x_train).mean(axis=0))",
        'pps' : """(pps.predictors(x_train.join(y_train), 
                                'target', sorted=False, output='df').model_score
                                )""", 
        'pearson' : "abs(x_train.join(y_train).corr(method='spearman').target[:-1])",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target[:-1])",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target[:-1])"
    }

In [12]:
x_train, y_train = make_regression(n_samples=10000, n_features=100, 
                                       n_informative=2,
                                       shuffle=False,
                                       random_state=0
                                      )
x_train = pd.DataFrame(x_train) 
y_train = pd.DataFrame(y_train)
y_train = y_train.rename({0:'target'}, axis=1)

m = ensemble.RandomForestRegressor(random_state=0, n_jobs=n_jobs, verbose=0)
m.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=0)

In [126]:
a = %timeit -o -n1 -r100 eval(imp_type_dict.get('gini'))

18.1 ms ± 2.3 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [130]:
a.timings

[0.01973970397375524,
 0.018892546999268234,
 0.01948619302129373,
 0.019193379004718736,
 0.019372512004338205,
 0.019454822002444416,
 0.021159576019272208,
 0.019719153991900384,
 0.01990813500015065,
 0.019389140012208372,
 0.023386665008729324,
 0.026463171001523733,
 0.021354077995056286,
 0.016844339988892898,
 0.017138392984634265,
 0.019410522014368325,
 0.016927669988945127,
 0.017465016018832102,
 0.016995731013594195,
 0.01720411400310695,
 0.019750394014408812,
 0.016794919996755198,
 0.021453108987770975,
 0.016894000000320375,
 0.016917052009375766,
 0.016862019983818755,
 0.016678849002346396,
 0.016852020024089143,
 0.017002342006890103,
 0.01689730101497844,
 0.01689349001389928,
 0.017011921998346224,
 0.016845779988216236,
 0.016619658999843523,
 0.017240083019714803,
 0.01700365199940279,
 0.016808009997475892,
 0.017202513001393527,
 0.01713580300565809,
 0.016543448000447825,
 0.0170553220086731,
 0.016718559025321156,
 0.016729038004996255,
 0.016542687022592872

In [113]:
b = np.array(a.all_runs)

In [86]:
imp_types = [
             'gini',
            'mutual_info'
            ]

In [133]:
# %%time`
for k in datasets.keys():
    for i in imp_types:
        x_train = datasets[k].data
        y_train = datasets[k].target
        print(f'executing {i}, for {k}')
        time_lst = %timeit -o -n1 -r100 eval(imp_type_dict.get(i))
        time_arr = np.array(time_lst)
        #%timeit -o -n1 -r100 eval(imp_type_dict.get('gini'))
        print(f'executed {i}')
#         print(f'mean exec_time = {time_arr.mean()}')

executing gini, for regr_features_10_samples_100_seed_0
18.7 ms ± 3.19 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed gini
executing mutual_info, for regr_features_10_samples_100_seed_0
11 ms ± 1.26 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed mutual_info
executing gini, for regr_features_10_samples_1000_seed_0
17.2 ms ± 770 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed gini
executing mutual_info, for regr_features_10_samples_1000_seed_0
41.1 ms ± 1.42 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed mutual_info
executing gini, for regr_features_10_samples_10000_seed_0
17.4 ms ± 1.23 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed gini
executing mutual_info, for regr_features_10_samples_10000_seed_0
500 ms ± 15.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
executed mutual_info
executing gini, for regr_features_100_samples_10_seed_0
The slowest run took 4.06 times longer than the fastes

KeyboardInterrupt: 

In [106]:
# subset = datasets['regr_features_100_samples_10000_seed_0']

In [107]:
df_res = get_stats(imp_types, models, datasets, debug=True, 
                   random_states=random_states, dataset_states=dataset_states)

Data state progress:   0%|          | 0/1 [00:00<?, ?it/s]

Model random state progress:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset progress:   0%|          | 0/1 [00:00<?, ?it/s]

Model progress for dataset 0:   0%|          | 0/3 [00:00<?, ?it/s]

Importance type progress for model 0:   0%|          | 0/9 [00:00<?, ?it/s]

Importance type progress for model 1:   0%|          | 0/9 [00:00<?, ?it/s]

Importance type progress for model 2:   0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
# export results to local dir as csv file
df_res.to_csv(f'regression_synthethic_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv', index=False)

In [14]:
# import seaborn as sns

# plot = sns.lineplot(x='iteration', y='mse_test', hue='imp_type',
#             data=df_res.query('model == "XGBRegressor" and dataset == "diabetes"')
#             )